<a href="https://colab.research.google.com/github/samuveljebakumar/fish/blob/main/Copy_of_today.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import os
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
DATASET_PATH = "/content/drive/MyDrive/today"

In [ ]:
def extract_mel_sequence(file_path):
    y, sr = librosa.load(file_path, duration=5, mono=True)
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    mel_db = librosa.power_to_db(mel, ref=np.max)

    # Convert to shape (time, features) for LSTM
    mel_db = mel_db.T     # now shape = (time_frames, 128)

    # Fix size (keep 300 frames)
    if mel_db.shape[0] < 300:
        pad_width = 300 - mel_db.shape[0]
        mel_db = np.pad(mel_db, ((0, pad_width), (0,0)), mode='constant')
    else:
        mel_db = mel_db[:300, :]

    return mel_db


In [ ]:
X = []
Y = []

# Add a check for DATASET_PATH existence
if not os.path.isdir(DATASET_PATH):
    raise FileNotFoundError(
        f"Dataset path not found: '{DATASET_PATH}'. "
        "Please ensure Google Drive is mounted and the path is correct."
    )

print(f"Starting data loading from: {DATASET_PATH}")
found_files_count = 0
found_labels_set = set() # To track actual labels found for files

# First, gather all unique true labels (e.g., bird names) across all subdirectories
unique_labels = set()
for split_folder in os.listdir(DATASET_PATH):
    split_path = os.path.join(DATASET_PATH, split_folder)
    if os.path.isdir(split_path):
        print(f"  Found split folder: {split_folder}")
        if not os.listdir(split_path):
            print(f"    Warning: Split folder '{split_folder}' is empty.")
        for label_folder_name in os.listdir(split_path):
            label_path = os.path.join(split_path, label_folder_name)
            if os.path.isdir(label_path):
                print(f"    Found label folder: {label_folder_name}")
                unique_labels.add(label_folder_name)
            else:
                print(f"    Skipping non-directory item in '{split_folder}': {label_folder_name}")
    else:
        print(f"  Skipping non-directory item in DATASET_PATH: {split_folder}")

# Sort labels to ensure consistent encoding
sorted_unique_labels = sorted(list(unique_labels))
print(f"\nUnique labels found (before filtering for actual files): {sorted_unique_labels}")
# The LabelEncoder will be applied in the next cell (0XaniC02UjC1)

# Second pass to extract melspectrograms and associate with string labels
for split_folder in os.listdir(DATASET_PATH):
    split_path = os.path.join(DATASET_PATH, split_folder)
    if os.path.isdir(split_path):
        for label_folder_name in os.listdir(split_path):
            label_folder_path = os.path.join(split_path, label_folder_name)
            if os.path.isdir(label_folder_path):
                if not os.listdir(label_folder_path):
                    print(f"      Warning: Label folder '{label_folder_name}' in '{split_folder}' is empty.")
                for file_name in os.listdir(label_folder_path):
                    if file_name.endswith(".wav") or file_name.endswith(".mp3") or file_name.endswith(".aac") or file_name.endswith(".flac") or file_name.endswith(".ogg") or file_name.endswith(".m4a"):
                        file_path = os.path.join(label_folder_path, file_name)
                        print(f"        Processing file: {file_path}")
                        try:
                            mel_seq = extract_mel_sequence(file_path)
                            X.append(mel_seq)
                            Y.append(label_folder_name) # Append the actual bird label string
                            found_files_count += 1
                            found_labels_set.add(label_folder_name)
                        except Exception as e:
                            print(f"        Error processing {file_name}: {e}")
                    else:
                        print(f"        Skipping non-audio file (unsupported extension or not audio): {file_name}")

X = np.array(X)
Y = np.array(Y)

print("\n--- Data Loading Summary ---")
print(f"Total files successfully processed: {found_files_count}")
print(f"Actual labels from processed files: {sorted(list(found_labels_set))}")
print("X shape:", X.shape)
print("Number of samples:", len(X))
print("Number of classes:", len(np.unique(Y)) if len(Y) > 0 else 0)
print("Classes:", np.unique(Y) if len(Y) > 0 else "No classes found")


### Next Steps:

Now that the `FileNotFoundError` is resolved, please run all the cells in the notebook from the beginning, starting with the import statements and Google Drive mount, then proceeding through data loading, preprocessing, model definition, training, evaluation, and finally the prediction. This will ensure the entire workflow is executed correctly.

In [ ]:
le = LabelEncoder()
Y_encoded = le.fit_transform(Y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y_encoded, test_size=0.2, random_state=42, stratify=Y_encoded)

In [ ]:
# Define the CRNN model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv1D, MaxPooling1D, Bidirectional, LSTM,
    Dense, Dropout, BatchNormalization
)

model = Sequential([
    Conv1D(64, 5, activation="relu", padding="same", input_shape=(300,128)),
    BatchNormalization(),
    MaxPooling1D(2),

    Conv1D(128, 5, activation="relu", padding="same"),
    BatchNormalization(),
    MaxPooling1D(2),

    Bidirectional(LSTM(128, return_sequences=False)),

    Dense(256, activation="relu"),
    Dropout(0.3),

    Dense(len(np.unique(Y_encoded)), activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

In [ ]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=16
)

In [ ]:
# Evaluate the model
loss, acc = model.evaluate(X_test, y_test)
print("CRNN Test Accuracy:", acc)

In [ ]:
# Final prediction
# --- FINAL PREDICTION BLOCK ---

from google.colab import files

# Using a different compatible MP3 file for prediction, as the previous one caused issues.
test_file = '/content/XC983831 - Asian Koel - Eudynamys scolopaceus.wav' # You might need to adjust this path to your test file

# Convert audio → mel sequence (same function used in training)
mel_seq = extract_mel_sequence(test_file)
mel_seq = mel_seq.reshape(1, 300, 128)

# Predict
pred = model.predict(mel_seq)
bird = le.inverse_transform([np.argmax(pred)])

print("\n================================")
print("🔥 FINAL BIRD SOUND PREDICTION 🔥")
print("Predicted Bird:", bird[0])
print("================================")